In [ ]:
!pwd

In [1]:
from lra.layers import *
from lra.models import *
from lra.setups import *
from lra.utils  import *

import sys
sys.path.append('/home/yorshula/long-range-arena')

from lra_benchmarks.listops.input_pipeline import get_datasets

In [2]:
train_dataset, valid_dataset, test_dataset, encoder = get_datasets(1, 'basic', data_dir='/home/yorshula/lra_release/listops-1000/', batch_size=LISTOPS_SETUP['batch_size'], max_length=LISTOPS_SETUP['max_length'])

INFO:tensorflow:/home/yorshula/lra_release/listops-1000/basic_train.tsv
INFO:tensorflow:/home/yorshula/lra_release/listops-1000/basic_val.tsv
INFO:tensorflow:/home/yorshula/lra_release/listops-1000/basic_test.tsv


2022-05-02 13:58:17.814034: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-02 13:58:17.816578: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-02 13:58:17.863772: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-02 13:58:17.866204: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-02 13:58:17.868213: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

INFO:tensorflow:Finished preprocessing
INFO:tensorflow:Building vocab
INFO:tensorflow:Processed 0
INFO:tensorflow:Processed 1000
INFO:tensorflow:Finished processing vocab size=15


In [6]:
LISTOPS_SETUP['device'] = 'cuda:1'
#LISTOPS_SETUP['model_type'] = ClassificationTransformerSkip

def att_factory(hidden_dim, qkv_dim, num_heads, dropout_rate):
    
    lka = nn.Sequential(
        AMGOLU(num_heads, qkv_dim, qkv_dim // num_heads // 4, dropout_rate, nn.Sigmoid(), nn.Identity(), False, LAMBDA=0.0),
        AMGOLU(num_heads, qkv_dim, qkv_dim // num_heads // 4, dropout_rate, nn.Sigmoid(), nn.Identity(), False, LAMBDA=0.0),
        AMGOLU(num_heads, qkv_dim, qkv_dim // num_heads // 4, dropout_rate, nn.Sigmoid(), nn.Softplus(), False, LAMBDA=0.0),
        
        #GatedOrthoKernel(num_heads, hidden_dim, dropout_rate, nn.Sigmoid(), nn.Identity(), False, LAMBDA=0.1),
        #GatedOrthoKernel(num_heads, hidden_dim, dropout_rate, nn.Sigmoid(), nn.Identity(), False, LAMBDA=0.1),
        #GatedOrthoKernel(num_heads, hidden_dim, dropout_rate, nn.Sigmoid(), nn.Softplus(), False, LAMBDA=0.1)

        #HeadWiseFF(num_heads, qkv_dim, dropout_rate, nn.Softplus(), use_bias=False, LAMBDA=0.1),   
    )
    return LKAAttention(hidden_dim, qkv_dim, num_heads, dropout_rate, lka)
    
    #return FtAttention(hidden_dim, qkv_dim, num_heads, dropout_rate)
    #return SimpleAttention(hidden_dim, qkv_dim, num_heads, dropout_rate, use_lin=True)
    #return SimpleAttention(hidden_dim, qkv_dim, num_heads, dropout_rate, use_lin=False)

model, criterion, optimizer, schedule_func, scheduler = training_setup(LISTOPS_SETUP, encoder.vocab_size, att_factory)

In [ ]:
import torch

LISTOPS_SETUP['device'] = 'cuda:1'
#LISTOPS_SETUP['steps'] = 20
#LISTOPS_SETUP['eval_period'] = 10

test_accuracy = [  ]

for i in range(1): ####!!!!!!!!!!!!!!
  path = 'model_to_test_' + str(i) + '.b'

  model, criterion, optimizer, schedule_func, scheduler = training_setup(LISTOPS_SETUP, encoder.vocab_size, att_factory)

  checkpoint = train_listops_model(LISTOPS_SETUP, model, path, train_dataset, valid_dataset, optimizer, criterion, scheduler)
  model.load_state_dict(checkpoint['model_state_dict'])
  
  _, _, acc = cls_test(model, criterion, test_dataset, LISTOPS_SETUP['device'])
  test_accuracy.append(acc)

test_accuracy = np.mean(test_accuracy)

print(f'\nTotal accuracy: {test_accuracy:.4f}')

Epoch 0
[====================] 10/10: - running_loss: 2.2850 - running_reg: 0.000000 - running_acc: 0.0948 - lr: 0.00000 - epoch_loss: 2.2835 - epoch_reg: 0.000000 - epoch_acc: 0.0938 - valid_loss: 2.2835 - valid_reg: 0.000000 - valid_acc: 0.1525 - epoch_time: 38.1596 s
Epoch 1
[=====>--------------] 3/10: - running_loss: 2.2465 - running_reg: 0.000000 - running_acc: 0.2068 - lr: 0.00000

In [5]:
model

ClassificationTransformer(
  (embed_layer): TEmbedding(
    (embedding): Embedding(17, 512, padding_idx=0)
  )
  (blocks): ModuleList(
    (0): TBlock(
      (layernorm_input): LayerNorm((512,), eps=1e-06, elementwise_affine=True)
      (layernorm_inter): LayerNorm((512,), eps=1e-06, elementwise_affine=True)
      (attention): LKAAttention(
        (q): Linear(in_features=512, out_features=512, bias=True)
        (k): Linear(in_features=512, out_features=512, bias=True)
        (v): Linear(in_features=512, out_features=512, bias=True)
        (lka): Sequential(
          (0): AMGOLU(
            (orth_weight): HWLinear()
            (gate_weight_a): HWLinear()
            (gate_weight_b): HWLinear()
            (kernel_nonlinearity): Identity()
            (gate_nonlinearity): Sigmoid()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): AMGOLU(
            (orth_weight): HWLinear()
            (gate_weight_a): HWLinear()
            (gate_weight_b): HWLinea